In [2]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 819 kB/s eta 0:00:01


In [3]:
!pip install bayesian-optimization

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11684 sha256=56360bcf19985b8fbd7880f4e6429da9d72d548b9dff9a19ea621f2a5ad58759
  Stored in directory: /home/tione/.cache/pip/wheels/fd/cc/65/a1df5517b6cbf53d499363834b1e5b069376f89de749ae567e
Successfully built bayesian-optimization


In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,MinMaxScaler
from bayes_opt import BayesianOptimization
import joblib
import time
np.random.seed(2020)


def lgb_model_age(learning_rate,subsample,colsample_bytree,num_leaves,reg_alpha,reg_lambda):
    params = {'objective':'multiclass',
              'num_class' : 10,
             'n_estimators':200}
    
    params['num_leaves'] = int(num_leaves)
    params['learning_rate'] = float(learning_rate)
    params['subsample'] = float(subsample)
    params['colsample_bytree'] = float(colsample_bytree)
    params['reg_alpha'] = float(reg_alpha)
    params['reg_lambda'] = float(reg_lambda)
    
    model = lgb.LGBMClassifier(**params)
    val = cross_val_score(model,data,label_age,cv = 5).mean()
    return val

def lgb_model_gender(learning_rate,subsample,colsample_bytree,num_leaves,reg_alpha,reg_lambda):
    params = {'objective':'binary',
             'n_estimators':200}
    
    params['num_leaves'] = int(num_leaves)
    params['learning_rate'] = float(learning_rate)
    params['subsample'] = float(subsample)
    params['colsample_bytree'] = float(colsample_bytree)
    params['reg_alpha'] = float(reg_alpha)
    params['reg_lambda'] = float(reg_lambda)
    
    model = lgb.LGBMClassifier(**params)
    val = cross_val_score(model,data,label_gender,cv = 5).mean()
    return val

def load_data():
    # user
    data = pd.read_csv('../w2v_feat_data/train_data.csv')
    data = data.head(10000)
    label = data[['age','gender']]
    
    data = data.drop(['user_id','age','gender'],axis = 1)
    return data,label

In [5]:
data,label = load_data()
label_gender = label['gender']
label_age = label['age']

In [15]:
param_dist_age = {
            'learning_rate':(0.01,0.5),
            'subsample':(0.1,0.9),
            'colsample_bytree':(0.1,0.9),
            'num_leaves' :(32,128),
            'reg_alpha' :(0,0.1),
            'reg_lambda' :(0,0.1)
            }

star = time.time()
age_bayes = BayesianOptimization(lgb_model_age, pbounds=param_dist_age)
age_bayes.maximize(init_points=5, n_iter=25)
print('age param  is \n',age_bayes.max)
end = time.time()
print('age time spend ',end-star)

|   iter    |  target   | colsam... | learni... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------
|  1        |  0.264    |  0.3386   |  0.06542  |  96.71    |  0.0922   |  0.04357  |  0.3792   |
|  2        |  0.262    |  0.286    |  0.3376   |  89.2     |  0.05591  |  0.03175  |  0.1343   |


KeyboardInterrupt: 

In [ ]:

param_dist_gender = {
            'learning_rate':(0.01,1),
            'subsample':(0.1,1),
            'colsample_bytree':(0.1,1),
            'num_leaves' :(16,256),
            'reg_alpha' :(0,0.1),
            'reg_lambda' :(0,0.1)
            }

star = time.time()
gender_bayes = BayesianOptimization(lgb_model_gender, pbounds=param_dist_gender)
gender_bayes.maximize(init_points=5, n_iter=25)
print('gender param  is \n',gender_bayes.max)
end = time.time()
print('gender time spend ',end-star)

print('******************************************************')

param_dist_age = {
            'learning_rate':(0.01,1),
            'subsample':(0.1,1),
            'colsample_bytree':(0.1,1),
            'num_leaves' :(16,256),
            'reg_alpha' :(0,0.1),
            'reg_lambda' :(0,0.1)
            }

star = time.time()
age_bayes = BayesianOptimization(lgb_model_age, pbounds=param_dist_age)
age_bayes.maximize(init_points=5, n_iter=25)
print('age param  is \n',age_bayes.max)
end = time.time()
print('age time spend ',end-star)

|   iter    |  target   | colsam... | learni... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------
|  1        |  0.8574   |  0.1904   |  0.531    |  216.6    |  0.05892  |  0.08801  |  0.8237   |
|  2        |  0.8609   |  0.6938   |  0.2668   |  199.8    |  0.05682  |  0.05845  |  0.2904   |
|  3        |  0.8497   |  0.9973   |  0.7138   |  229.7    |  0.03092  |  0.0264   |  0.887    |
|  4        |  0.8473   |  0.643    |  0.7242   |  163.1    |  0.09727  |  0.05135  |  0.3809   |
|  5        |  0.8612   |  0.6014   |  0.2263   |  112.7    |  0.09983  |  0.07735  |  0.4112   |
|  6        |  0.8581   |  0.8827   |  0.434    |  113.5    |  0.01788  |  0.07476  |  0.4985   |
|  7        |  0.8633   |  0.1684   |  0.136    |  16.09    |  0.06741  |  0.01441  |  0.4821   |
|  8        |  0.8588   |  0.2326   |  0.02121  |  57.25    |  0.0543   |  0.08776  |  0.1767   |
|  9        |  0.857